# Projet Kayak
# Module de récupération (via APIs) des informations de localisation des villes et des informations méteo
#

## Imports des modules externes

In [1]:
import pandas as pd 
import requests

## Paramètres de contexte du projet

In [2]:
# top-35 des villes à visiter en France - Périmètre du projet

cities_list = [    
    "Mont Saint Michel",
    "St Malo",
    "Bayeux",
    "Le Havre",
    "Rouen",
    "Paris",
    "Amiens",
    "Lille",
    "Strasbourg",
    "Chateau du Haut Koenigsbourg",
    "Colmar",
    "Eguisheim",
    "Besancon",
    "Dijon",
    "Annecy",
    "Grenoble",
    "Lyon",
    "Gorges du Verdon",
    "Bormes les Mimosas",
    "Cassis",
    "Marseille",
    "Aix en Provence",
    "Avignon",
    "Uzes",
    "Nimes",
    "Aigues Mortes",
    "Saintes Maries de la mer",
    "Collioure",
    "Carcassonne",
    "Ariege",
    "Toulouse",
    "Montauban",
    "Biarritz",
    "Bayonne",
    "La Rochelle"]




## Récupération des latitudes and longitudes des villes depuis l'API de openstreetmap

In [3]:
get_geoloc_api = False # récupération des geoloc de l'API ou lecture d'une précédente récupération


if get_geoloc_api :

    df_cities_geoloc = pd.DataFrame({"city_id" : range(len(cities_list)), "city_name" : cities_list}) 
    lon_list = []
    lat_list = []

    for city in cities_list :
       playload = {"city" : city, "format" : "json"}
       r = requests.get("https://nominatim.openstreetmap.org/search", params = playload)

       lon_list.append(float(r.json()[0]["lon"]))
       lat_list.append(float(r.json()[0]["lat"]))
        
       time.sleep(2) # pour respet des limites de l'usge "free" de l'API

    df_cities_geoloc["city_lon"] = lon_list
    df_cities_geoloc["city_lat"] = lat_list

    df_cities_geoloc.to_csv("./data/df_cities_geoloc.csv")

else :
     
     df_cities_geoloc = pd.read_csv("./data/df_cities_geoloc_save.csv", index_col = 0)

# display(df_cities_geoloc.describe())
# display(df_cities_geoloc.head(100))

## Récupération de la méteo des villes via l'API openweathermap

In [4]:
# Test de fonctionnement de l'API avec un appel simple

# playload = {"lat" : 48.858890, "lon" : 2.320041, "units" : "metric", "lang" : "fr", "APPID" : "d137b1197bdc9dcc93ccf7a2ce365618"}
# r = requests.get("http://api.openweathermap.org/data/2.5/forecast", params = playload)
# display(r)

<Response [200]>

In [4]:
# la méteo future est donnée pour une latitude et logitude pour les 5 jours à venir par intervalle de 3 heures
# Les indicateurs recueillis seront consolidés pour définir la meteo globale d'une ville pour les 5 jors à venir
# le choix et les crières de consolidation des indicateurs pourront être adaptées 


get_weather_api = False # récupération de la meteo de l'API ou lecture d'une précédente récupération

if get_weather_api :

    df_weather = pd.DataFrame(columns = ["city_id", "dt", "dt_txt", "temp", "temp_min", "temp_max", "pressure", "humidity", "w_main", "w_desc", "w_icon", "wind_speed", "pop"])  

    for c in df_cities_geoloc.index :

        city_id = df_cities_geoloc.loc[c, "city_id"]
        lon = df_cities_geoloc.loc[c, "city_lon"]
        lat = df_cities_geoloc.loc[c, "city_lat"]

        playload = {"lat" : lat, "lon" : lon, "units" : "metric", "lang" : "fr", "APPID" : "d137b1197bdc9dcc93ccf7a2ce365618"}
        r = requests.get("http://api.openweathermap.org/data/2.5/forecast", params = playload)

        for item in r.json()["list"] :

            record = []
            record.append(city_id)
            record.append(item["dt"])
            record.append(item["dt_txt"])
            record.append(item["main"]["temp"])
            record.append(item["main"]["temp_min"])
            record.append(item["main"]["temp_max"])
            record.append(item["main"]["pressure"])
            record.append(item["main"]["humidity"])
            record.append(item["weather"][0]["main"])
            record.append(item["weather"][0]["description"])
            record.append(item["weather"][0]["icon"])
            record.append(item["wind"]["speed"])
            record.append(item["pop"])

            df_weather.loc[len(df_weather)] = record


        df_weather.to_csv("./data/df_weather.csv")

else :
     
     df_weather = pd.read_csv("./data/df_weather_save.csv", index_col = 0)

print(len(df_weather))
display(df_weather.describe())
display(df_weather.head(100))

1400


,city_id,dt,temp,temp_min,temp_max,pressure,humidity,wind_speed,pop
count,1400.000000,1.400000e+03,1400.000000,1400.000000,1400.000000,1400.000000,1400.000000,1400.000000,1400.000000
mean,17.000000,1.728729e+09,14.559279,14.477993,14.562907,1012.990714,77.543571,3.317671,0.182486
std,10.103114,1.247132e+05,3.978014,3.958139,3.976734,6.593739,13.718849,2.259128,0.347296
min,0.000000,1.728518e+09,4.020000,4.020000,4.020000,977.000000,32.000000,0.030000,0.000000
25%,8.000000,1.728624e+09,11.810000,11.747500,11.810000,1011.000000,69.000000,1.690000,0.000000
50%,17.000000,1.728729e+09,14.720000,14.595000,14.720000,1015.000000,80.000000,2.750000,0.000000
75%,26.000000,1.728834e+09,17.652500,17.610000,17.652500,1017.000000,89.000000,4.252500,0.200000
max,34.000000,1.728940e+09,24.580000,24.580000,24.580000,1023.000000,100.000000,11.810000,1.000000


,city_id,dt,dt_txt,temp,temp_min,temp_max,pressure,humidity,w_main,w_desc,w_icon,wind_speed,pop
0,0,1728518400,2024-10-10 00:00:00,13.51,12.55,13.51,997,94,Rain,légère pluie,10n,3.98,1.00
1,0,1728529200,2024-10-10 03:00:00,13.31,12.91,13.31,998,93,Rain,légère pluie,10n,4.49,1.00
2,0,1728540000,2024-10-10 06:00:00,12.66,12.23,12.66,1001,90,Rain,légère pluie,10n,2.96,1.00
3,0,1728550800,2024-10-10 09:00:00,13.35,13.35,13.35,1006,85,Rain,légère pluie,10d,1.72,1.00
4,0,1728561600,2024-10-10 12:00:00,15.72,15.72,15.72,1008,68,Rain,légère pluie,10d,2.68,0.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2,1728680400,2024-10-11 21:00:00,11.28,11.28,11.28,1014,80,Rain,légère pluie,10n,3.34,0.27
96,2,1728691200,2024-10-12 00:00:00,10.82,10.82,10.82,1013,86,Clouds,couvert,04n,3.80,0.00
97,2,1728702000,2024-10-12 03:00:00,9.86,9.86,9.86,1011,95,Clouds,couvert,04n,3.49,0.00
98,2,1728712800,2024-10-12 06:00:00,10.90,10.90,10.90,1011,88,Clouds,couvert,04n,2.50,0.00


## Calcul du score de la qualité des destinations
Score par ville en fct de la temperature et de la pluie moyenne prévus (à titre d'exemple) pour les prochains jours

In [5]:
df_cities = df_cities_geoloc.copy()
df_cities["temp"] = df_weather.groupby("city_id")["temp"].mean()
df_cities["pop"] = df_weather.groupby("city_id")["pop"].mean()

df_cities.to_csv("./data/df_cities.csv")